In [ ]:
import boto3
s3 = boto3.client('s3')
bucket = 'sagemaker-eu-west-1-366243680492'
s3_file = 'data-processing-2020-06-12-08-37-01-419/output/preprocessed/MRPC/train.tsv'

s3.download_file(bucket, s3_file, 'train.tsv')

In [1]:
import sagemaker
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
endpoint_name = 'training-pipeline-2020-06-12-10-23-45'

class Predictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=json_serializer, 
                         deserializer=json_deserializer, content_type='application/json')

bert_predictor = Predictor(endpoint_name, sagemaker_session=sagemaker_session)

In [ ]:
data = pd.read_csv('train.tsv', sep='\t', error_bad_lines=False)

In [ ]:
payload = data['#1 String'][0]

In [ ]:
item

In [ ]:
# results = bert_predictor.predict(item)
# print(results)

In [ ]:
# !pip install pytorch_pretrained_bert

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(modelpath)

text = "dummy. although he had already eaten a large meal, he was still very hungry."
target = "hungry"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = tokenized_text.index(target)
tokenized_text[masked_index] = '[MASK]'

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [1] * len(tokenized_text)
# this is for the dummy first sentence. 
segments_ids[0] = 0
segments_ids[1] = 0

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained(modelpath)
model.eval()

# Predict all tokens
predictions = model(tokens_tensor, segments_tensors)
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])

print("Original:", text)
print("Masked:", " ".join(tokenized_text))

print("Predicted token:", predicted_token)
print("Other options:")
# just curious about what the next few options look like.
for i in range(10):
    predictions[0,masked_index,predicted_index] = -11100000
    predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print(predicted_token)